In [3]:
# Stationery shop dataset
dataset = [
    ['Pen', 'Notebook', 'Eraser', 'Sharpener'],
    ['Pencil', 'Notebook', 'Ruler', 'Marker'],
    ['Pen', 'Highlighter', 'Notebook'],
    ['Stapler', 'Paper Clips', 'Glue', 'Notebook'],
    ['Pen', 'Ruler', 'Sharpener', 'Notebook', 'Marker'],
    ['Eraser', 'Sticky Notes', 'Glue'],
    ['Pen', 'Notebook', 'Marker', 'Stapler'],
    ['Pencil', 'Eraser', 'Sharpener', 'Glue'],
    ['Highlighter', 'Notebook', 'Paper Clips'],
    ['Pen', 'Notebook', 'Pencil', 'Eraser']
]

In [4]:
# FP-Tree Node Class
class FPTreeNode:
    def __init__(self, item, count):
        self.item = item
        self.count = count
        self.children = {}

In [5]:
# Pre-processing to filter transactions
def pre_process(transactions, min_support):
    support = {}
    for transaction in transactions:
        for item in transaction:
            support[item] = support.get(item, 0) + 1

    # Sorting items based on frequency
    sorted_support = dict(sorted(support.items(), key=lambda item: item[1], reverse=True))

    # Removing items below min support
    filtered_support = {k: v for k, v in sorted_support.items() if v >= min_support}

    # Sorting transactions based on filtered support
    processed_list = []
    for transaction in transactions:
        filtered_transaction = [item for item in transaction if item in filtered_support]
        processed_list.append(sorted(filtered_transaction, key=lambda x: filtered_support[x], reverse=True))

    return processed_list

In [6]:
# Function to create FP-Tree
def create_tree(root, processed_list, i, j):
    if j >= len(processed_list[i]):
        return
    item = processed_list[i][j]
    if item in root.children:
        root.children[item].count += 1
    else:
        root.children[item] = FPTreeNode(item, 1)
    
    create_tree(root.children[item], processed_list, i, j + 1)

In [7]:
# Function to print FP-Tree
def print_tree(node, indent=0):
    if node.item is not None:
        print("|  " * indent + f"-{node.item} ({node.count})")
    for child in node.children.values():
        print_tree(child, indent + 1)

In [8]:
# Function to construct and display FP-Tree
def construct_FPTree(transactions, min_support):
    processed_list = pre_process(transactions, min_support)
    root = FPTreeNode("Root", 0)

    for i in range(len(processed_list)):
        create_tree(root, processed_list, i, 0)
    
    print_tree(root)

In [9]:
# Run FP-Tree on Stationery Dataset
construct_FPTree(dataset, 2)

-Root (0)
|  -Notebook (8)
|  |  -Pen (5)
|  |  |  -Eraser (2)
|  |  |  |  -Sharpener (1)
|  |  |  |  -Pencil (1)
|  |  |  -Highlighter (1)
|  |  |  -Sharpener (1)
|  |  |  |  -Marker (1)
|  |  |  |  |  -Ruler (1)
|  |  |  -Marker (1)
|  |  |  |  -Stapler (1)
|  |  -Pencil (1)
|  |  |  -Marker (1)
|  |  |  |  -Ruler (1)
|  |  -Glue (1)
|  |  |  -Stapler (1)
|  |  |  |  -Paper Clips (1)
|  |  -Highlighter (1)
|  |  |  -Paper Clips (1)
|  -Eraser (2)
|  |  -Glue (1)
|  |  -Pencil (1)
|  |  |  -Sharpener (1)
|  |  |  |  -Glue (1)


In [10]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

In [11]:

# Stationery shop dataset
dataset = [
    ['Pen', 'Notebook', 'Eraser', 'Sharpener'],
    ['Pencil', 'Notebook', 'Ruler', 'Marker'],
    ['Pen', 'Highlighter', 'Notebook'],
    ['Stapler', 'Paper Clips', 'Glue', 'Notebook'],
    ['Pen', 'Ruler', 'Sharpener', 'Notebook', 'Marker'],
    ['Eraser', 'Sticky Notes', 'Glue'],
    ['Pen', 'Notebook', 'Marker', 'Stapler'],
    ['Pencil', 'Eraser', 'Sharpener', 'Glue'],
    ['Highlighter', 'Notebook', 'Paper Clips'],
    ['Pen', 'Notebook', 'Pencil', 'Eraser']
]

In [12]:
# Convert dataset into a format suitable for FP-Growth
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [13]:
# Apply FP-Growth algorithm
min_support = 0.2  # Minimum support threshold (adjustable)
frequent_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)

In [14]:
# Display frequent itemsets
print("Frequent Itemsets:")
print(frequent_itemsets)

Frequent Itemsets:
    support                    itemsets
0       0.8                  (Notebook)
1       0.5                       (Pen)
2       0.4                    (Eraser)
3       0.3                 (Sharpener)
4       0.3                    (Pencil)
5       0.3                    (Marker)
6       0.2                     (Ruler)
7       0.2               (Highlighter)
8       0.3                      (Glue)
9       0.2                   (Stapler)
10      0.2               (Paper Clips)
11      0.5             (Pen, Notebook)
12      0.2               (Eraser, Pen)
13      0.2          (Eraser, Notebook)
14      0.2     (Eraser, Pen, Notebook)
15      0.2         (Eraser, Sharpener)
16      0.2            (Pen, Sharpener)
17      0.2       (Notebook, Sharpener)
18      0.2  (Pen, Notebook, Sharpener)
19      0.2          (Pencil, Notebook)
20      0.2            (Eraser, Pencil)
21      0.3          (Notebook, Marker)
22      0.2               (Pen, Marker)
23      0.2     (Pen,

In [15]:
# Generate association rules
min_confidence = 0.5  # Minimum confidence threshold (adjustable)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

In [16]:
# Printing only the association rules
print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


Association Rules:
              antecedents         consequents  support  confidence      lift
0                   (Pen)          (Notebook)      0.5    1.000000  1.250000
1              (Notebook)               (Pen)      0.5    0.625000  1.250000
2                (Eraser)               (Pen)      0.2    0.500000  1.000000
3                (Eraser)          (Notebook)      0.2    0.500000  0.625000
4           (Eraser, Pen)          (Notebook)      0.2    1.000000  1.250000
5      (Eraser, Notebook)               (Pen)      0.2    1.000000  2.000000
6                (Eraser)     (Pen, Notebook)      0.2    0.500000  1.000000
7                (Eraser)         (Sharpener)      0.2    0.500000  1.666667
8             (Sharpener)            (Eraser)      0.2    0.666667  1.666667
9             (Sharpener)               (Pen)      0.2    0.666667  1.333333
10            (Sharpener)          (Notebook)      0.2    0.666667  0.833333
11       (Pen, Sharpener)          (Notebook)      0.2  